In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip")
test_df =  pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip")
result_df = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/sample_submission.csv.zip")
train_df["color"].replace({"clear": (1/6), "white": (2/6),"blood":(3/6),"green":(4/6),"blue":(5/6),"black": 1 }, inplace=True)
test_df["color"].replace({"clear": (1/6), "white": (2/6),"blood":(3/6),"green":(4/6),"blue":(5/6),"black": 1 }, inplace=True)

In [ ]:
euDistArray = []
for index,testMonster in test_df.iterrows():
    euDistArray.append(np.sqrt((train_df.bone_length-testMonster.bone_length)**2+( train_df.rotting_flesh-testMonster.rotting_flesh )**2+(train_df.hair_length-testMonster.hair_length  )**2
    +(train_df.has_soul-testMonster.has_soul  )**2+ (train_df.color-testMonster.color  )**2))

In [ ]:
#for each monster of the test set we will return K indexes pointing in the monsters of the train set
def findKnnIndexes(K,euDistance):
    kneighbors=[]
    for  i in range(K):
        kneighbors.append(euDistance.index(min(euDistance)))
        euDistance[euDistance.index(min(euDistance))]=999.99
    return kneighbors

def findMostFrequent(types):
    counters = {'Ghost':0 , 'Goblin':0 , 'Ghoul':0}
    for i in types:
        counters[i]+=1
    return(max(counters, key=counters.get))

In [ ]:
def predictMonsterType(K):
    results = []
    for i in euDistArray:
        knnIndexes = findKnnIndexes(K,i.tolist())
        monsterTypes = []
        for j in knnIndexes:
            monsterTypes.append(train_df.type[j])
        results.append(findMostFrequent(monsterTypes))
        monsterTypes.clear()
    return results

In [ ]:
K = 10
result_df.type = predictMonsterType(K)
print(result_df)

In [ ]:
filename = "submission.csv"
result_df.to_csv(filename,index = False)